In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.nn as nn

In [ ]:
X = np.load('C:\Data\Datageddon/X.npy')  
y = np.load('C:\Data\Datageddon/Y.npy')  

In [ ]:
print(X.shape, y.shape)

In [ ]:
# You may change the train-test split according to your will
X_train, y_train = X[:20000], y[:20000]
X_val, y_val = X[20000:], y[20000:]
#_val variables are basically test data variables


X_train = torch.from_numpy(X_train).float()  
y_train = torch.from_numpy(y_train).long()   
X_val = torch.from_numpy(X_val).float()  
y_val = torch.from_numpy(y_val).long().reshape(-1, 1)

In [ ]:
#batch size is the number of attributes processed until the model is updated
batch_size = 32

#Dataloader is used to load our data while Dataset is used to wrap an iterable around it to allow easy access
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
#Actual implementation of deep neural network(3 layered DL)
#Activation function used is relu i.e max(0,x)
import torch.nn.functional as F
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.fc1=nn.Linear(64*3*3*3,512)
    self.fc2=nn.Linear(512,64)
    self.fc3=nn.Linear(64,4)


  def forward(self,x):
    x=x.view(-1,64*3*3*3).clone()
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x
model=Net()
print(model)

In [ ]:
#We are optimizing our DL network and defining loss function.
import torch.optim as optim
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters())

In [ ]:
#This is the code for training our neural network. I am printing the loss for each epoch to see that the loss is reducing.
i=1
for epoch in range(20):
    train_loss = 0.
    for local_batch, local_labels in train_dataloader:
        y_pred = model(local_batch)
        loss = loss_fn(y_pred,local_labels)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_loss/=batch_size
    print(f"Epoch {i} loss : ", train_loss)
    i+=1
    


In [ ]:
# Accuracy of training dataset
acc = 0.
count = 0
with torch.set_grad_enabled(False):
  for local_batch, local_labels in train_dataloader:
    #predict the label
    y_pred = model(local_batch)
    #find the index of the element with max weight which will give us the label.
    y_pred = torch.argmax(y_pred, dim = 1)
    #then we check if that label is same as the given label(for accuracy measurements)

    for i in range(local_labels.shape[0]):
      if local_labels[i] == y_pred[i]:
        acc = 1./(count+1) + acc*count/(count + 1)
      else:
        acc = acc*count/(count + 1)
      count += 1
    
print(f"Accuracy {acc*100}")

In [ ]:
# Accuracy of validation dataset
acc = 0.
count = 0
with torch.set_grad_enabled(False):
  for local_batch, local_labels in val_dataloader:
    y_pred = model(local_batch)
    y_pred = torch.argmax(y_pred, dim = 1)

    for i in range(local_labels.shape[0]):
      if local_labels[i] == y_pred[i]:
        acc = 1./(count+1) + acc*count/(count + 1)
      else:
        acc = acc*count/(count + 1)
      count += 1
    
print(f"Accuracy {acc*100}")

In [ ]:
#saving the weights and other parameters for the model
#Since I was working on google colab, the file path is as below. If you want to save on your local machine, comment out the existing path and use the other file path
filepath='/content/drive/MyDrive/Datageddon_files/weights.pt'
# filepath='Datageddon/weights.pt'
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, filepath)